# Mudanças nas faixas do Imposto de Renda Pessoa Física do Brasil

Ninguém gosta de pagar imposto. Ninguém. Ponto. Nós gostamos dos benefícios, de viver em uma sociedade funcional, mas não de pagar a conta. E é por isso que em um país com serviços e infraestrutura capengas pagar um imposto alto passa a mesma sensação de pagar 20 reais de covert para um Voz e Violão de Jorge Vercilo num McDonalds. E é por isso que qualquer tentativa de reduzir [ou aumentar] a quantidade de impostos é recebida com muita alegria [ou raiva].

Logo, não é surpresa para ninguém que o [Projeto de Lei do Senado n° 517 de 2015](https://www25.senado.leg.br/web/atividade/materias/-/materia/122597) chame tanta atenção. Ele propõe a alteração da tabela de cobrança do imposto de renda para pessoa física da seguinte forma:  

In [ ]:
import pandas as pd

pl2015_taxbrackets = pd.read_csv("PL2015_TaxBrackets.csv",index_col=False)

pl2015_taxbrackets.style.format({'renda_mensal_min': '{:0,.2f}', 'renda_mensal_max': '{:0,.2f}', 'aliquota': '{:.0%}'})\
                  .bar(subset=['aliquota'], color='#32cd32')

Tab. 1: Imposto de renda do PL de 2015. É legal como não precisamos nos preocupar com álgebra de ponto flutuante ou intervalos abertos e fechados quando lidamos com valores monetários.

Como [98% do Brasil ganha até nove mil reais por mês](https://www.nexojornal.com.br/interativo/2016/01/11/O-seu-sal%C3%A1rio-diante-da-realidade-brasileira), o projeto viralizou. Para o jovem, a felicidade do primeiro salário só é reduzida pelo seu enquadramento na [tabela atual do imposto de renda](http://idg.receita.fazenda.gov.br/acesso-rapido/tributos/irpf-imposto-de-renda-pessoa-fisica#dedu--o-mensal-por-dependente%20conorf):

In [ ]:
current_taxbrackets = pd.read_csv("Current_TaxBrackets.csv",index_col=False)

current_taxbrackets.style.format({'renda_mensal_min': '{:0,.2f}', 'renda_mensal_max': '{:0,.2f}', 'aliquota': '{:.1%}'})\
                  .bar(subset=['aliquota'], color='#32cd32')

Tab.2 : Imposto de Renda atual. Gente, custava arredondar esses números aí?

O que não viralizou, por motivos óbvios, é o impacto dessa proposta de tabela na arrecadação da união. Mesmo que você defenda estado mínimo, estado máximo ou qualquer combinação linear dos dois extremos, mudanças bruscas não são legais. Existem três grandes conjuntos de leis ([Lei de Diretrizes Orçamentárias](https://pt.wikipedia.org/wiki/Lei_de_diretrizes_or%C3%A7ament%C3%A1rias), [Lei Orçamentária Anual](https://pt.wikipedia.org/wiki/Lei_or%C3%A7ament%C3%A1ria_anual) e a [Lei de Responsabilidade Fiscal](https://pt.wikipedia.org/wiki/Lei_de_Responsabilidade_Fiscal)) que podem ser resumidas na seguinte frase:

"O dinheiro tem que dar para pagar as contas."
-- Mãe do autor, 2005.

O PL da nova tabela não vem com uma análise sobre esse impacto, somente com a frase: "A redução do tributo devido promovida para as classes mais baixas será compensada com o aumento para as superiores", o que desperta em mim um grande sentimento de "citation needed". 

A Comissâo de Assuntos Econômicos (CAE) percebeu isso também quando se [manifestou](https://legis.senado.leg.br/sdleg-getter/documento?dm=7304671&disposition=inline) sobre o PL e solicitou um relatorio para a Consultoria de Orçamentos, Fiscalização e Controle do Senado Federal (CONORF), que possui esse nome fantástico mas uma sigla nem tanto. O parecer final da CAE contém a estimativa final da CONORF de R$ 250.92 bilhões de arrecadação para o ano de 2018 com a nova tabela. 

Não li o relatório da CONORF porque ele não foi disponibilizado no site deles (ou eu não sei usar aquele layout bizantino) nem anexado à tramitação do PL. Existem considerações muito fortes ness tipo de análise de impacto, logo não consigo acreditar nesses 250 bi assim, de mão beijada. Na verdade, não consigo nem comparar esse número com o atual já que nas [projeções oficiais](http://www.planejamento.gov.br/assuntos/orcamento-1/orcamentos-anuais/2018/ploa-2018/volume-i.pdf) a arrecadação com Pessoa Física será de 31.8 bi, e não parece ser esse número que a CONORF calculou. Olhando a projeção para a arrecadação completa também não ajuda, já que o total fica em 342 bi, ja contando com o Imposto de Renda das empresas (outro buraco).

Vou tentar aqui então reproduzir a análise e chegar em algo que possamos comparar. É claro que esse tipo de análise deveria ser feita pela Receita Federal, que *realmente* tem a renda mensal das pessoas. Uns dois SQL resolveriam o problema (ou um com sub-query, mas tem gente que não curte então respeito). 

## Impacto na arrecadação, primeira tentativa

Vou primeiro estimar quantas pessoas existem em cada uma das faixas. A População Economicamente Ativa do Brasil está estimada em [63%](https://brasilescola.uol.com.br/geografia/populacao-economicamente-ativa-pea.htm) da [população total](https://www.google.com.br/search?q=populacao+do+brasil+em+2015&oq=populacao+do+brasil+em+2015). 

In [ ]:
PEA = 206000000*0.6305
PEA

Para estimar o valor arrecadado em um determinado ano pela tabela atual sem a lista de pessoas e seus salários preciso assumir uma distribuição de pessoas entre as faixas. Por hora, vamos considerar, muito errôneamente, que essa distriuição é uniforme. 

In [ ]:
current_bracket_total_revenue = current_taxbrackets.copy()
current_bracket_total_revenue['pagantes'] = PEA/5.0

Se a distribuição de pessoas é uniforme basta então encontrarmos o ponto médio de cada faixa para calcularmos o valor arrecadado. A última faixa no entanto é um problema. Ela não tem um valor superior. Aqui, vou considerar esse valor como sendo o do [salário do cargo de presidente da república](http://www2.camara.leg.br/camaranoticias/noticias/POLITICA/504248-CAMARA-APROVA-REDUCAO-DE-SALARIO-DE-PRESIDENTE-DA-REPUBLICA-E-MINISTROS.html) em 2015, só pela piada.

In [ ]:
current_bracket_total_revenue['renda_mensal_max'][4] =  30934.70 

current_bracket_total_revenue['renda_mensal_media'] = (current_bracket_total_revenue['renda_mensal_min'] + 
                                                       current_bracket_total_revenue['renda_mensal_max'])/2.0
def total_revenue(bracket_df):
    temp_df = bracket_df.copy()
    temp_df['valor_declarado_faixa'] = temp_df['pagantes']*temp_df['renda_mensal_media']
    temp_df['valor_pago_faixa'] = 13.33*temp_df['valor_declarado_faixa']*temp_df['aliquota']
    
    return sum(temp_df['valor_pago_faixa'])

print("Valor total arrecadado: {:0,.2f}".format(total_revenue(current_bracket_total_revenue)))

Um valor muuito maior do que o normal arrecadado pela receita. Mas vamos parar de nos preocupar com a realidade por um instante e repetir o mesmo raciocínio para a nova tabela. Aqui, a última faixa tem como limite superior o salário de três presidentes:

In [ ]:
pl2015_taxbrackets_total_revenue = pl2015_taxbrackets
pl2015_taxbrackets_total_revenue['pagantes'] = PEA/9.0

pl2015_taxbrackets_total_revenue.iloc[8,1] =  30934.70*3  # três presidentes!

pl2015_taxbrackets_total_revenue['renda_mensal_media'] = (pl2015_taxbrackets_total_revenue['renda_mensal_min'] + 
                                                         pl2015_taxbrackets_total_revenue['renda_mensal_max'])/2.0

print("Valor total arrecadado: {:0,.2f}".format(total_revenue(pl2015_taxbrackets_total_revenue)))

Nessa estimativa bem simplista a nova tabela resulta em uma arrecadação quase 5x maior. Mas nossa premissa de distribuição uniforme da população por faixa de renda não é nenhum pouco razoável. Nâo existem tantos ricos assim. 

Vamos complicar um pouco as coisas e assumir que, começando da faixa mais rica, cada faixa for 12 vezes maior do que a faixa anterior:

In [ ]:
population_richest_bracket = PEA/(sum([10**i for i in range(0,5)]))
current_bracket_total_revenue['pagantes'] = [population_richest_bracket*(10**i) for i in range(0,5)][::-1]

print("Pagantes : {:0,.2f}".format(sum(current_bracket_total_revenue['pagantes'])))
print("Tabela atual arrecadado:  {:0,.2f}".format(total_revenue(current_bracket_total_revenue)))

current_bracket_total_revenue[['renda_mensal_media','pagantes']]

Ambos valor total arrecadado e distribuição de pagantes nas faixas não parecem muito feios para mim. Vou então redistribuir essas pessoas na tabela nova. Para isso, vou assumir que, dentro das faixas, a população se mantém de forma uniforme. 

Como as novas faixas não coincidem com as atuais, temos que fazer um pouco de ginástica para conseguir a quantidade de pagantes na nova tabela: 

In [ ]:
pagantes_nova_tabela = {}

pagantes_nova_tabela[0] =  (pl2015_taxbrackets_total_revenue['renda_mensal_media'][0]/current_bracket_total_revenue['renda_mensal_max'][0])\
                           *current_bracket_total_revenue['pagantes'][0]
    
pagantes_nova_tabela[1] =  current_bracket_total_revenue['pagantes'][0] - pagantes_nova_tabela[0] +\
                           current_bracket_total_revenue['pagantes'][1] +\
                           (pl2015_taxbrackets_total_revenue['renda_mensal_media'][1]-current_bracket_total_revenue['renda_mensal_min'][2])\
                           /(current_bracket_total_revenue['renda_mensal_max'][2]-current_bracket_total_revenue['renda_mensal_min'][2])\
                           *current_bracket_total_revenue['pagantes'][2]

pagantes_nova_tabela[2] =  (1-(pl2015_taxbrackets_total_revenue['renda_mensal_media'][1]-current_bracket_total_revenue['renda_mensal_min'][2])\
                           /(current_bracket_total_revenue['renda_mensal_max'][2]-current_bracket_total_revenue['renda_mensal_min'][2])\
                           )*current_bracket_total_revenue['pagantes'][2]+\
                           current_bracket_total_revenue['pagantes'][3] +\
                           (pl2015_taxbrackets_total_revenue['renda_mensal_media'][2]-current_bracket_total_revenue['renda_mensal_min'][4])\
                           /(current_bracket_total_revenue['renda_mensal_max'][4]-current_bracket_total_revenue['renda_mensal_min'][4])\
                           *current_bracket_total_revenue['pagantes'][4]

for i in range(3,9):
    pagantes_nova_tabela[i] = ((1-(pl2015_taxbrackets_total_revenue['renda_mensal_media'][2]-current_bracket_total_revenue['renda_mensal_min'][4])\
        /(current_bracket_total_revenue['renda_mensal_max'][4]-current_bracket_total_revenue['renda_mensal_min'][4])\
        )/6)*current_bracket_total_revenue['pagantes'][4]

pl2015_taxbrackets_total_revenue['pagantes'] = [pagantes_nova_tabela[i] for i in range(0,9)]

print("Pagantes : {:0,.2f}".format(sum(pl2015_taxbrackets_total_revenue['pagantes'])))
print("Tabela nova arrecadado:  {:0,.2f}".format(total_revenue(pl2015_taxbrackets_total_revenue)))
pl2015_taxbrackets_total_revenue

Logo, a tabela do PL resulta em uma arrecadação quase o dobro da atual. Mas não vamos sejamos precipitados, assumir uma distribuição uniforme de pessoas dentro das faixas é uma premissa muito forte. Vamos então investigar isso com pouco mais de detalhes.

## Impacto na arrecadação, agora com PNAD

A Pesquisa Nacional por Amostra de Domicílios (PNAD) mede umas coisas muito legais. Você consegue achar números de como as pessoas se dividem em termos de localidade, escolaridade, uso de tecnologias e, mais importante que isso, rendimento. Os últimos dados que o IBGE publicou são de [2015](https://ww2.ibge.gov.br/home/estatistica/populacao/trabalhoerendimento/pnad2015/sintese_defaultxls.shtm), mas tudo bem, o PL é de 2015 também. Vamos trabalhar com esse ano daqui pra frente. Você consegue baixar o arquivo zip com todos as planilhas da pesquisa, mas a única que interessa pra gente é a PNAD_2015_07rendimento_xls.zip\07rendimento\02ocupadas\tab7_2_1.xls:

In [ ]:
pnad_2015 = pd.read_csv("PNAD2015_tab7_2_1.csv",index_col=False)
print("Pagantes : {:0,.2f}".format(sum(pnad_2015['pagantes'])))

pnad_2015.style.format({'renda_mensal_min': '{:0,.2f}', 'renda_mensal_max': '{:0,.2f}', 'pagantes': '{:0,.2f}'})\
                  .bar(subset=['pagantes'], color='#32cd32')

A PNAD nos diz que a quantidade de pessoas ocupadas, e que pagariam imposto, é menor do que a PEA, com um pico entre 1 e 2 salários mínimos de 2015. Podemos ver então qual o impacto na quantidade de pessoas isentas.

In [ ]:
pnad_2015_modified = pnad_2015
pnad_2015_modified['renda_mensal_max'][7] =  30934.70*3

pnad_2015_modified['renda_mensal_media'] = (pnad_2015_modified['renda_mensal_min']+\
                                                      pnad_2015_modified['renda_mensal_max'])/2.0

pnad_2015_modified['renda_mensal_media']

In [ ]:
print("Pessoas isentas na Tabela atual : {:0,.2f}".format(sum([pnad_2015_modified['pagantes'][0] +\
                                                               pnad_2015_modified['pagantes'][1] +\
                                                               pnad_2015_modified['pagantes'][2] +\
                                                               pnad_2015_modified['pagantes'][3]/2.0])))
print("Pessoas isentas na Tabela nova :  {:0,.2f}".format(sum([pnad_2015_modified['pagantes'][0] +\
                                                               pnad_2015_modified['pagantes'][1] +\
                                                               pnad_2015_modified['pagantes'][2] +\
                                                               pnad_2015_modified['pagantes'][3] +\
                                                               pnad_2015_modified['pagantes'][4]/2.0])))

O que nos leva a quase 10 milhões de pessoas a mais com isenção de imposto na nova tabela. 

Vamos agora calcular a arrecadação da tabela atual de acordo com os dados da PNAD.

In [ ]:
import numpy as np

current_bracket_revenue_pnad = pnad_2015_modified.copy()

aliquotas = []
for renda_media in pnad_2015_modified['renda_mensal_media']:
    for i, row in current_taxbrackets.iterrows(): 
        if renda_media >= row['renda_mensal_min'] and (renda_media < row['renda_mensal_max'] or np.isnan(row['renda_mensal_max'])):
            aliquotas.append(row['aliquota'])
            
current_bracket_revenue_pnad['aliquota'] = aliquotas
current_bracket_revenue_pnad['pagantes'][3] = current_bracket_revenue_pnad['pagantes'][3]/2.0

In [ ]:
print("Pagantes : {:0,.2f}".format(sum(current_bracket_revenue_pnad['pagantes'])))
print("Tabela atual arrecadado:  {:0,.2f}".format(total_revenue(current_bracket_revenue_pnad)))
current_bracket_revenue_pnad[['renda_mensal_min','renda_mensal_max','pagantes','aliquota']]\
                  .style.format({'renda_mensal_min': '{:0,.2f}', 'renda_mensal_max': '{:0,.2f}', 'pagantes': '{:0,.2f}'})\
                  .bar(subset=['pagantes'], color='#32cd32')\
                  .bar(subset=['aliquota'], color='#45ed42')
        

Uou. Provalvemente errado. A questão aqui é que a PNAD não está preocupada em discernir o que é rendimento de tributável de outras coisas estranhas, como recebimento de lucros e dividendos que é como autonomos e empresários declaram seus salários. 
Além de rendimentos não-tributáveis, a economia brasileira contém uma grande parcela de informalidade e corrupção. 

Vou considerar então que todo cidadão tem uma probabilidade *D* de desviar, de forma legal ou ilegal, o seu pagamento de imposto. Para encontrar o valor de *D*, calculo o total arrecadado até que ele seja próximo do [informado pela receita em 2015](http://www.valor.com.br/brasil/4403702/arrecadacao-soma-r-12-trilhao-em-2015-pior-resultado-em-cinco-anos): 

In [ ]:
Ds = []
Revenues = []

for D in range(0,100):
    
    tempdf = current_bracket_revenue_pnad.copy()
    
    tempdf['pagantes'] = (1.0-D/100.0)*tempdf['pagantes'] 
    revenue = total_revenue(tempdf)
    
    Ds.append(D/100.0)
    Revenues.append(revenue)

In [ ]:
%matplotlib inline
import seaborn as sns

sns.set(rc={'figure.figsize':(11.7,4)})
sns.set_style("whitegrid")

df = pd.DataFrame({'D' : Ds, 'Revenue' : Revenues})

df['Real Revenue'] = 29699000000

ax = sns.regplot(x=df["D"], y=df["Real Revenue"],fit_reg=False,label="Real")
sns.regplot(x=df["D"], y=df["Revenue"],ax = ax, fit_reg=False,label="Simulated")

ax.set(xlim=(0, 1))
ax.set(ylim=(0, 4e11))
ax.set_xticks([0,0.5,0.75,0.85,0.92,1.0])
ax.legend()

In [ ]:
D = 0.92
current_bracket_revenue_pnad['pagantes'] = current_bracket_revenue_pnad['pagantes']*(1-D)

print("Tabela atual arrecadado:  {:0,.2f}".format(total_revenue(current_bracket_revenue_pnad)))

current_bracket_revenue_pnad[['renda_mensal_min','renda_mensal_max','pagantes','aliquota']]\
                  .style.format({'renda_mensal_min': '{:0,.2f}', 'renda_mensal_max': '{:0,.2f}', 'pagantes': '{:0,.2f}'})\
                  .bar(subset=['pagantes'], color='#32cd32')\
                  .bar(subset=['aliquota'], color='#45ed42')

O que nos dá um belo e desconfortável valor para D de 93%. O que é como dizer que na média 93% da renda de uma pessoa não é tributável. Com o que sabemos de corrupção no brasil, esperava um valor de D alto, mas não tão alto.

Por hora, vamos assumir que a metodologia é correta e replicar então a PNAD e D na nova tabela:

In [ ]:
pl2015_bracket_revenue_pnad = pnad_2015_modified.copy()

aliquotas = []
for renda_media in pnad_2015_modified['renda_mensal_media']:
    for i, row in pl2015_taxbrackets.iterrows():        
        if renda_media >= row['renda_mensal_min'] and (renda_media < row['renda_mensal_max'] or np.isnan(row['renda_mensal_max'])):
            aliquotas.append(row['aliquota'])
    
pl2015_bracket_revenue_pnad['aliquota'] = aliquotas

pl2015_bracket_revenue_pnad['pagantes'][4] = pl2015_bracket_revenue_pnad['pagantes'][4]/2.0
pl2015_bracket_revenue_pnad['aliquota'][4] = pl2015_taxbrackets['aliquota'][1]

In [ ]:
pl2015_bracket_revenue_pnad['pagantes'] = pl2015_bracket_revenue_pnad['pagantes']*(1.0-D)

In [ ]:
print("Tabela nova arrecadado:  {:0,.2f}".format(total_revenue(pl2015_bracket_revenue_pnad)))
pl2015_bracket_revenue_pnad[['renda_mensal_min','renda_mensal_max','pagantes','aliquota']]\
                  .style.format({'renda_mensal_min': '{:0,.2f}', 'renda_mensal_max': '{:0,.2f}', 'pagantes': '{:0,.2f}'})\
                  .bar(subset=['pagantes'], color='#32cd32')\
                  .bar(subset=['aliquota'], color='#45ed42')

A nova tabela então retira muita gente das faixas pagantes e não compensa com o aumento de impostos dos mais ricos. A arrecadação chega a cair quase 7 bilhões nesse novo esquema. 

O problema desse tipo de análise, mais uma vez, são os mais ricos. Não é razoável assumir uma distribuição uniforme de pessoas entre os riquíssimos. Isso pode estar criando distorções que prejudicam a arrecadação da tabela nova. Podemos resolver isso assumindo algumas coisas a mais.

## Impacto sob influência do hit Xibom bombom

"E o destino todo mundo ja conhece: é que o de cima sobe e o debaixo desce."
-- Meninas, As. Hit Xibom bombom, circa 1999